In [1]:
import pandas as pd, numpy as np
import itertools, requests, json

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [5]:
dc=dc.set_index('uid')
dm=dm.set_index('uid')
lm=lm.set_index('uid')

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.merge(dc).drop_duplicates()

In [8]:
dl=df.merge(lm)

Working df ready. Now infer location.

In [9]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [10]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [11]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [12]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [13]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,0,0,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Marosvásárhely', 'Jedd', 'Koronka', 'Székely...",Marosvásárhely,46.5386,24.5514,"Marosvásárhely, 0"
1,0,0,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Marosvásárhely', 'Jedd', 'Koronka', 'Székely...",Jedd,46.549,24.629,"Jedd, 0"
2,0,0,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Marosvásárhely', 'Jedd', 'Koronka', 'Székely...",Koronka,46.5206,24.6189,"Koronka, 0"
3,0,0,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/fenykeppel-nyom...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Marosszék', 'medve']","Fényképpel nyomatékosítják, csak óvatosan, mer...",...,1,1,0,0,"['Marosvásárhely', 'Marosvásárhely', 'Koronka'...","['Marosvásárhely', 'Jedd', 'Koronka', 'Székely...",Székelybós,46.5314,24.6698,"Székelybós, 0"
4,1,1,2019-05-02,2019. május 02.,https://szekelyhon.ro/aktualis/celkeresztbe-ke...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Udvarhelyszék']",Célkeresztbe kerülhet a Szentábrahámon juhokat...,...,1,0,0,0,Szentábrahám,Szentábrahám,Szentábrahám,46.3347,25.0091,"Szentábrahám, 1"
5,2,2,2019-05-14,2019. május 14.,https://szekelyhon.ro/aktualis/korabbi-elohely...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,['Háromszék'],Korábbi élőhelyén engedték szabadon a tavaly e...,...,1,0,0,0,"['Sepsibükszád', 'Haraly', 'Haraly', 'Sepsibük...",Sepsibükszád,Sepsibükszád,46.0077,25.8411,"Sepsibükszád, 2"
6,3,3,2019-05-15,2019. május 15.,https://szekelyhon.ro/aktualis/medve-tamadt-eg...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Csíkszék', 'medve']",Medve támadt egy madarasi juhászra,...,1,3,0,0,Csíkmadaras,Csíkmadaras,Csíkmadaras,46.4932,25.7458,"Csíkmadaras, 3"
7,4,4,2019-05-15,2019. május 15.,https://szekelyhon.ro/aktualis/szurkulettel-jo...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Marosszék', 'medve']",Szürkülettel jövő félelem: hárombocsos medvétő...,...,1,1,0,0,NaN,Bárdos,Bárdos,,,"Bárdos, 4"
8,5,5,2019-05-15,2019. május 15.,https://szekelyhon.ro/aktualis/faklyaval-igyek...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Udvarhelyszék']","Fáklyával igyekeznek elriasztani a medvét, amí...",...,1,0,0,0,Szentábrahám,Szentábrahám,Szentábrahám,46.3347,25.0091,"Szentábrahám, 5"
9,6,6,2019-05-24,2019. május 24.,https://szekelyhon.ro/aktualis/tovabballt-az-a...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Udvarhelyszék']",Továbbállt az agyagdombi medvecsalád,...,1,0,0,0,Székelyudvarhely,Székelyudvarhely,Székelyudvarhely,46.305,25.2926,"Székelyudvarhely, 6"


Save data for loading in viz app

In [14]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [15]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [16]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')